In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=2, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
 - 5s - loss: 1.9472 - acc: 0.2542
Epoch 2/100
 - 0s - loss: 1.9358 - acc: 0.2699
Epoch 3/100
 - 0s - loss: 1.8642 - acc: 0.2844
Epoch 4/100
 - 0s - loss: 1.8792 - acc: 0.2934
Epoch 5/100
 - 0s - loss: 1.8038 - acc: 0.2788
Epoch 6/100
 - 0s - loss: 1.8185 - acc: 0.2676
Epoch 7/100
 - 0s - loss: 1.7966 - acc: 0.2665
Epoch 8/100
 - 0s - loss: 1.8362 - acc: 0.2576
Epoch 9/100
 - 0s - loss: 1.7567 - acc: 0.2889
Epoch 10/100
 - 0s - loss: 1.7469 - acc: 0.2587
Epoch 11/100
 - 0s - loss: 1.7388 - acc: 0.2665
Epoch 12/100
 - 0s - loss: 1.7438 - acc: 0.2811
Epoch 13/100
 - 0s - loss: 1.7326 - acc: 0.2441
Epoch 14/100
 - 0s - loss: 1.7470 - acc: 0.2822
Epoch 15/100
 - 0s - loss: 1.7208 - acc: 0.2632
Epoch 16/100
 - 0s - loss: 1.6205 - acc: 0.3337
Epoch 17/100
 - 0s - loss: 1.6794 - acc: 0.2878
Epoch 18/100
 - 0s - loss: 1.6785 - acc: 0.2979
Epoch 19/100
 - 0s - loss: 1.6225 - acc: 0.3091
Epoch 20/100
 - 0s - loss: 1.5785 - acc: 0.3068
Epoch 21/100
 - 0s - loss: 1.5969 - acc: 0.3035
E

 - 0s - loss: 1.4575 - acc: 0.3337
Epoch 73/100
 - 0s - loss: 1.4155 - acc: 0.3505
Epoch 74/100
 - 0s - loss: 1.3170 - acc: 0.4065
Epoch 75/100
 - 0s - loss: 1.4447 - acc: 0.3483
Epoch 76/100
 - 0s - loss: 1.3985 - acc: 0.3415
Epoch 77/100
 - 0s - loss: 1.4105 - acc: 0.3270
Epoch 78/100
 - 0s - loss: 1.4057 - acc: 0.3505
Epoch 79/100
 - 0s - loss: 1.4094 - acc: 0.3382
Epoch 80/100
 - 0s - loss: 1.4268 - acc: 0.3707
Epoch 81/100
 - 0s - loss: 1.3427 - acc: 0.3729
Epoch 82/100
 - 0s - loss: 1.3541 - acc: 0.3662
Epoch 83/100
 - 0s - loss: 1.3381 - acc: 0.3606
Epoch 84/100
 - 0s - loss: 1.4002 - acc: 0.3628
Epoch 85/100
 - 0s - loss: 1.3218 - acc: 0.3852
Epoch 86/100
 - 0s - loss: 1.4001 - acc: 0.3415
Epoch 87/100
 - 0s - loss: 1.3605 - acc: 0.3897
Epoch 88/100
 - 0s - loss: 1.3517 - acc: 0.3583
Epoch 89/100
 - 0s - loss: 1.3133 - acc: 0.3684
Epoch 90/100
 - 0s - loss: 1.3319 - acc: 0.3953
Epoch 91/100
 - 0s - loss: 1.3766 - acc: 0.3796
Epoch 92/100
 - 0s - loss: 1.3369 - acc: 0.3830
Epoch

Epoch 44/100
 - 0s - loss: 0.3902 - acc: 0.8365
Epoch 45/100
 - 0s - loss: 0.3794 - acc: 0.8611
Epoch 46/100
 - 0s - loss: 0.4297 - acc: 0.8320
Epoch 47/100
 - 0s - loss: 0.2799 - acc: 0.8959
Epoch 48/100
 - 0s - loss: 0.2888 - acc: 0.8847
Epoch 49/100
 - 0s - loss: 0.2934 - acc: 0.8925
Epoch 50/100
 - 0s - loss: 0.3053 - acc: 0.8746
Epoch 51/100
 - 0s - loss: 0.3310 - acc: 0.8891
Epoch 52/100
 - 0s - loss: 0.2851 - acc: 0.8891
Epoch 53/100
 - 0s - loss: 0.3046 - acc: 0.8858
Epoch 54/100
 - 0s - loss: 0.3064 - acc: 0.8869
Epoch 55/100
 - 0s - loss: 0.2570 - acc: 0.9082
Epoch 56/100
 - 0s - loss: 0.2557 - acc: 0.9071
Epoch 57/100
 - 0s - loss: 0.3169 - acc: 0.8746
Epoch 58/100
 - 0s - loss: 0.2942 - acc: 0.8947
Epoch 59/100
 - 0s - loss: 0.2336 - acc: 0.9082
Epoch 60/100
 - 0s - loss: 0.1857 - acc: 0.9395
Epoch 61/100
 - 0s - loss: 0.2404 - acc: 0.8970
Epoch 62/100
 - 0s - loss: 0.2715 - acc: 0.9104
Epoch 63/100
 - 0s - loss: 0.2004 - acc: 0.9138
Epoch 64/100
 - 0s - loss: 0.1884 - acc:

 - 0s - loss: 0.9426 - acc: 0.5906
Epoch 16/100
 - 0s - loss: 0.9661 - acc: 0.5951
Epoch 17/100
 - 0s - loss: 0.9397 - acc: 0.5996
Epoch 18/100
 - 0s - loss: 1.0147 - acc: 0.5593
Epoch 19/100
 - 0s - loss: 0.8913 - acc: 0.6197
Epoch 20/100
 - 0s - loss: 0.8422 - acc: 0.6320
Epoch 21/100
 - 0s - loss: 0.8273 - acc: 0.6499
Epoch 22/100
 - 0s - loss: 0.8731 - acc: 0.6320
Epoch 23/100
 - 0s - loss: 0.8121 - acc: 0.6600
Epoch 24/100
 - 0s - loss: 0.7571 - acc: 0.6846
Epoch 25/100
 - 0s - loss: 0.7586 - acc: 0.6846
Epoch 26/100
 - 0s - loss: 0.8854 - acc: 0.6723
Epoch 27/100
 - 0s - loss: 0.8541 - acc: 0.6499
Epoch 28/100
 - 0s - loss: 0.6853 - acc: 0.7282
Epoch 29/100
 - 0s - loss: 0.6745 - acc: 0.7237
Epoch 30/100
 - 0s - loss: 0.6587 - acc: 0.7248
Epoch 31/100
 - 0s - loss: 0.5971 - acc: 0.7595
Epoch 32/100
 - 0s - loss: 0.6239 - acc: 0.7550
Epoch 33/100
 - 0s - loss: 0.5153 - acc: 0.7953
Epoch 34/100
 - 0s - loss: 0.5144 - acc: 0.7931
Epoch 35/100
 - 0s - loss: 0.7220 - acc: 0.7539
Epoch

 - 0s - loss: 0.1105 - acc: 0.9608
Epoch 87/100
 - 0s - loss: 0.1100 - acc: 0.9563
Epoch 88/100
 - 0s - loss: 0.1367 - acc: 0.9552
Epoch 89/100
 - 0s - loss: 0.1174 - acc: 0.9608
Epoch 90/100
 - 0s - loss: 0.1166 - acc: 0.9541
Epoch 91/100
 - 0s - loss: 0.1053 - acc: 0.9608
Epoch 92/100
 - 0s - loss: 0.1054 - acc: 0.9642
Epoch 93/100
 - 0s - loss: 0.0995 - acc: 0.9653
Epoch 94/100
 - 0s - loss: 0.1071 - acc: 0.9653
Epoch 95/100
 - 0s - loss: 0.1096 - acc: 0.9586
Epoch 96/100
 - 0s - loss: 0.1073 - acc: 0.9642
Epoch 97/100
 - 0s - loss: 0.1086 - acc: 0.9586
Epoch 98/100
 - 0s - loss: 0.0984 - acc: 0.9653
Epoch 99/100
 - 0s - loss: 0.0807 - acc: 0.9776
Epoch 100/100
 - 0s - loss: 0.0943 - acc: 0.9619
Epoch 1/100
 - 7s - loss: 1.9051 - acc: 0.2912
Epoch 2/100
 - 0s - loss: 3.5256 - acc: 0.2744
Epoch 3/100
 - 0s - loss: 6.7530 - acc: 0.2699
Epoch 4/100
 - 0s - loss: 4.3469 - acc: 0.3527
Epoch 5/100
 - 0s - loss: 4.7405 - acc: 0.2598
Epoch 6/100
 - 0s - loss: 1.6391 - acc: 0.3975
Epoch 7/10

...

Epoch 72/100
 - 0s - loss: 0.1799 - acc: 0.9362
Epoch 73/100
 - 0s - loss: 0.1835 - acc: 0.9283
Epoch 74/100
 - 0s - loss: 0.1666 - acc: 0.9451
Epoch 75/100
 - 0s - loss: 0.1662 - acc: 0.9462
Epoch 76/100
 - 0s - loss: 0.1635 - acc: 0.9418
Epoch 77/100
 - 0s - loss: 0.1691 - acc: 0.9384
Epoch 78/100
 - 0s - loss: 0.1713 - acc: 0.9395
Epoch 79/100
 - 0s - loss: 0.1259 - acc: 0.9530
Epoch 80/100
 - 0s - loss: 0.1366 - acc: 0.9530
Epoch 81/100
 - 0s - loss: 0.1343 - acc: 0.9608
Epoch 82/100
 - 0s - loss: 0.1340 - acc: 0.9530
Epoch 83/100
 - 0s - loss: 0.1350 - acc: 0.9474
Epoch 84/100
 - 0s - loss: 0.1564 - acc: 0.9429
Epoch 85/100
 - 0s - loss: 0.1160 - acc: 0.9608
Epoch 86/100
 - 0s - loss: 0.1136 - acc: 0.9675
Epoch 87/100
 - 0s - loss: 0.1425 - acc: 0.9541
Epoch 88/100
 - 0s - loss: 0.1284 - acc: 0.9563
Epoch 89/100
 - 0s - loss: 0.1175 - acc: 0.9642
Epoch 90/100
 - 0s - loss: 0.1189 - acc: 0.9630
Epoch 91/100
 - 0s - loss: 0.1274 - acc: 0.9586
Epoch 92/100
 - 0s - loss: 0.1161 - acc:

Epoch 43/100
 - 0s - loss: 0.6677 - acc: 0.7122
Epoch 44/100
 - 0s - loss: 0.6205 - acc: 0.7469
Epoch 45/100
 - 0s - loss: 0.6527 - acc: 0.7268
Epoch 46/100
 - 0s - loss: 0.6298 - acc: 0.7324
Epoch 47/100
 - 0s - loss: 0.6261 - acc: 0.7301
Epoch 48/100
 - 0s - loss: 0.6068 - acc: 0.7604
Epoch 49/100
 - 0s - loss: 0.5590 - acc: 0.7682
Epoch 50/100
 - 0s - loss: 0.5981 - acc: 0.7525
Epoch 51/100
 - 0s - loss: 0.5945 - acc: 0.7436
Epoch 52/100
 - 0s - loss: 0.5507 - acc: 0.7682
Epoch 53/100
 - 0s - loss: 0.5304 - acc: 0.7850
Epoch 54/100
 - 0s - loss: 0.5350 - acc: 0.7906
Epoch 55/100
 - 0s - loss: 0.5014 - acc: 0.7973
Epoch 56/100
 - 0s - loss: 0.5142 - acc: 0.7996
Epoch 57/100
 - 0s - loss: 0.4729 - acc: 0.8197
Epoch 58/100
 - 0s - loss: 0.4772 - acc: 0.8141
Epoch 59/100
 - 0s - loss: 0.5048 - acc: 0.7996
Epoch 60/100
 - 0s - loss: 0.4273 - acc: 0.8331
Epoch 61/100
 - 0s - loss: 0.4361 - acc: 0.8331
Epoch 62/100
 - 0s - loss: 0.4447 - acc: 0.8208
Epoch 63/100
 - 0s - loss: 0.4142 - acc:

Epoch 14/100
 - 0s - loss: 1.0603 - acc: 0.5112
Epoch 15/100
 - 0s - loss: 1.0671 - acc: 0.5157
Epoch 16/100
 - 0s - loss: 1.0061 - acc: 0.5391
Epoch 17/100
 - 0s - loss: 0.9533 - acc: 0.5783
Epoch 18/100
 - 0s - loss: 0.9991 - acc: 0.5347
Epoch 19/100
 - 0s - loss: 0.9830 - acc: 0.5727
Epoch 20/100
 - 0s - loss: 0.9828 - acc: 0.5470
Epoch 21/100
 - 0s - loss: 0.9476 - acc: 0.5604
Epoch 22/100
 - 0s - loss: 0.9337 - acc: 0.5783
Epoch 23/100
 - 0s - loss: 0.9037 - acc: 0.5906
Epoch 24/100
 - 0s - loss: 0.9058 - acc: 0.6096
Epoch 25/100
 - 0s - loss: 0.9065 - acc: 0.5996
Epoch 26/100
 - 0s - loss: 0.8774 - acc: 0.6074
Epoch 27/100
 - 0s - loss: 0.8603 - acc: 0.6219
Epoch 28/100
 - 0s - loss: 0.8335 - acc: 0.6275
Epoch 29/100
 - 0s - loss: 0.8434 - acc: 0.6130
Epoch 30/100
 - 0s - loss: 0.8204 - acc: 0.6365
Epoch 31/100
 - 0s - loss: 0.8046 - acc: 0.6611
Epoch 32/100
 - 0s - loss: 0.8049 - acc: 0.6387
Epoch 33/100
 - 0s - loss: 0.7317 - acc: 0.6913
Epoch 34/100
 - 0s - loss: 0.7510 - acc:

Epoch 85/100
 - 0s - loss: 0.1114 - acc: 0.9664
Epoch 86/100
 - 0s - loss: 0.1114 - acc: 0.9597
Epoch 87/100
 - 0s - loss: 0.0969 - acc: 0.9630
Epoch 88/100
 - 0s - loss: 0.0990 - acc: 0.9586
Epoch 89/100
 - 0s - loss: 0.1015 - acc: 0.9630
Epoch 90/100
 - 0s - loss: 0.0964 - acc: 0.9619
Epoch 91/100
 - 0s - loss: 0.0933 - acc: 0.9675
Epoch 92/100
 - 0s - loss: 0.0950 - acc: 0.9664
Epoch 93/100
 - 0s - loss: 0.0795 - acc: 0.9698
Epoch 94/100
 - 0s - loss: 0.0820 - acc: 0.9686
Epoch 95/100
 - 0s - loss: 0.0877 - acc: 0.9664
Epoch 96/100
 - 0s - loss: 0.0764 - acc: 0.9686
Epoch 97/100
 - 0s - loss: 0.0802 - acc: 0.9720
Epoch 98/100
 - 0s - loss: 0.0857 - acc: 0.9686
Epoch 99/100
 - 0s - loss: 0.0762 - acc: 0.9698
Epoch 100/100
 - 0s - loss: 0.0726 - acc: 0.9731
Epoch 1/100
 - 12s - loss: 1.9521 - acc: 0.2284
Epoch 2/100
 - 0s - loss: 2.1124 - acc: 0.3337
Epoch 3/100
 - 0s - loss: 2.5508 - acc: 0.3214
Epoch 4/100
 - 0s - loss: 1.9382 - acc: 0.3371
Epoch 5/100
 - 0s - loss: 1.6975 - acc: 0.

Epoch 56/100
 - 0s - loss: 0.1307 - acc: 0.9553
Epoch 57/100
 - 0s - loss: 0.1347 - acc: 0.9541
Epoch 58/100
 - 0s - loss: 0.1401 - acc: 0.9519
Epoch 59/100
 - 0s - loss: 0.1316 - acc: 0.9519
Epoch 60/100
 - 0s - loss: 0.1413 - acc: 0.9474
Epoch 61/100
 - 0s - loss: 0.1402 - acc: 0.9485
Epoch 62/100
 - 0s - loss: 0.1555 - acc: 0.9474
Epoch 63/100
 - 0s - loss: 0.1322 - acc: 0.9463
Epoch 64/100
 - 0s - loss: 0.1523 - acc: 0.9474
Epoch 65/100
 - 0s - loss: 0.1301 - acc: 0.9553
Epoch 66/100
 - 0s - loss: 0.1485 - acc: 0.9530
Epoch 67/100
 - 0s - loss: 0.1377 - acc: 0.9553
Epoch 68/100
 - 0s - loss: 0.1315 - acc: 0.9609
Epoch 69/100
 - 0s - loss: 0.1172 - acc: 0.9597
Epoch 70/100
 - 0s - loss: 0.1143 - acc: 0.9586
Epoch 71/100
 - 0s - loss: 0.1108 - acc: 0.9586
Epoch 72/100
 - 0s - loss: 0.1458 - acc: 0.9463
Epoch 73/100
 - 0s - loss: 0.1081 - acc: 0.9642
Epoch 74/100
 - 0s - loss: 0.1123 - acc: 0.9620
Epoch 75/100
 - 0s - loss: 0.0990 - acc: 0.9676
Epoch 76/100
 - 0s - loss: 0.1159 - acc:

In [9]:
t = grid_result.best_estimator_

In [13]:
print(t.score(X_test, y_test))

0.6875


In [ ]:
# t.model.save('../model_gs/keras_fs.h5')

In [ ]:
# tf.set_random_seed(42)

# # make keras model
# start = time.time()
# inputs = Input(shape=(X_train.shape[1],))

# x = Dense(512, activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# predictions = Dense(
#     len(df['Subclass'].value_counts()), activation='softmax')(x)
# model = Model(inputs=inputs, outputs=predictions)

# # compile
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adamax(),
#     metrics=['accuracy']
# )

# epochs = 100
# batch_size = 1000
# es = EarlyStopping(monitor='val_loss', patience=20)

# history = model.fit(
#     X_train,
#     y_train_for_keras,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test_for_keras),
#     verbose=1,
#     callbacks=[
#         es,
#     ]
# )

# model.save('../model/Keras_fs_pos.h5')

In [ ]:
load_model('../model/Keras_fs_pos.h5').evaluate(X_test, y_test_for_keras, verbose=0)[1]